# 0. Import Libraries

In [ ]:
import pandas as pd
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 1. Process Data

In [ ]:
def transform_and_calculate(file_path, stop_distance=2, rr_ratio=3, lookahead=30):
    # 1. Data transformation
    def load_and_transform(file_path):
        # Load CSV with proper datetime handling
        df = pd.read_csv(
            file_path,
            delimiter='\t',
            parse_dates={'datetime': ['<DATE>', '<TIME>']},
            date_parser=lambda x: pd.to_datetime(x, format='%Y.%m.%d %H:%M:%S'),
            usecols=['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>', '<SPREAD>']
        )
        
        # Clean column names
        df.columns = df.columns.str.strip('<>').str.lower()
        
        # Column renaming and reorganization
        df = df.rename(columns={'tickvol': 'vol'})
        df = df[['datetime', 'open', 'high', 'low', 'close', 'vol', 'spread']]
        
        return df.sort_values('datetime').reset_index(drop=True)

    # 2. Spread-adjusted condition calculation
    def calculate_condition_met(df, stop_distance, rr_ratio, lookahead):
        # Initialize both condition columns
        df['long_condition'] = False
        df['short_condition'] = False
        
        for i in range(len(df)):
            current = df.iloc[i]
            future = df.iloc[i+1:i+lookahead+1]
            
            if future.empty:
                break
            
            # Calculate entry prices with spread adjustment
            long_entry = current['close'] + current['spread'] * 0.01
            short_entry = current['close'] - current['spread'] * 0.01
            
            # Calculate price levels with spread
            long_stop = long_entry - stop_distance
            long_target = long_entry + (stop_distance * rr_ratio)
            short_stop = short_entry + stop_distance
            short_target = short_entry - (stop_distance * rr_ratio)
            
            # Track both scenarios
            long_met, long_broken = False, False
            short_met, short_broken = False, False
            
            for _, candle in future.iterrows():
                # Check long condition
                if not long_met and not long_broken:
                    if candle['low'] <= long_stop:
                        long_broken = True
                    elif candle['high'] >= long_target:
                        long_met = True
                
                # Check short condition
                if not short_met and not short_broken:
                    if candle['high'] >= short_stop:
                        short_broken = True
                    elif candle['low'] <= short_target:
                        short_met = True
                
                # Early exit if both scenarios resolved
                if (long_met or long_broken) and (short_met or short_broken):
                    break
            
            # Set conditions independently
            df.at[i, 'long_condition'] = long_met
            df.at[i, 'short_condition'] = short_met
        
        return df

    # Execute processing pipeline
    df = load_and_transform(file_path)
    df = calculate_condition_met(df, stop_distance, rr_ratio, lookahead)
    return df

# Usage example:
csv_path = 'Data/MT5/XAUUSD_M15_202012070900_202502282345.csv'
processed_data = transform_and_calculate(csv_path)
processed_data.to_csv('processed.csv', index=False)
print("Data processing complete.")

/tmp/ipykernel_9641/4249613357.py:5: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
/tmp/ipykernel_9641/4249613357.py:5: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(


Data processing complete with spread-adjusted conditions.
